In [4]:
#Parameters of MC estimation:

import numpy as np
import math

S_0 = 100. #Initial stock price
r = 0.05 #Risk-free rate
sig = 0.2 #Stock price volatility
K = 100. #Strike price
lam_0 = 0.01 #Intensity of departure process (before vesting)
lam = 0.04 #Intensity of departure process (after vesting)

T = 10. #Time to expiry
T_0 = 3. #Time until vesting
N = 2**10 #Number of sub-intervals of [0,T]
dt = T/float(N) #Length of each sub-interval
N_0 = int(T_0/dt) #Number of sub-intervals in vesting period

L_0 = 150 #Initial level of barrier
alpha = -0.02 #Decay parameter of barrier
L = L_0*np.exp(alpha*np.linspace(0, T, N))

M = 1000 #Number of Monte Carlo samples

####################################################

MC = np.zeros(M)
MC_a = np.zeros(M)

for m in range(0, M):

    Z = np.sqrt(dt)*np.random.normal(0, 1, N)

    S = np.zeros(N)
    S[0] = S_0

    t_i = 'NULL'
    S_ti = 'NULL'

    t_j = 'NULL'
    S_tj = 'NULL'

    t_k = 'NULL'
    S_tk = 'NULL'


    for i in range(0, N_0):

        P = np.random.poisson(lam_0*dt, 1)

        if P >= 1:
            t_i = 1
            break

        else:
            S[i+1] = S[i]*np.exp( (r - 0.5*(sig**2))*dt + sig*Z[i] )


    for i in range(N_0, N-1):

        P = np.random.poisson(lam*dt, 1)

        if P >= 1:
            t_j = i*dt
            S_tj = S[i]
            break

        elif S[i] > L[i]:
        #elif S[i] > K*np.exp(alpha*(i*dt - T)) Uncomment to use K_t barrier
            t_k = i*dt
            S_tk = S[i]
            break

        else:
            S[i+1] = S[i]*np.exp( (r - 0.5*(sig**2))*dt + sig*Z[i] )

    if t_i != 'NULL' and t_j == 'NULL' and t_k == 'NULL':
        V1 = 0

    if t_j != 'NULL' and t_i == 'NULL' and t_k == 'NULL':
        V1 = np.exp(-r*t_j)*np.maximum(S_tj - K, 0)

    if t_k != 'NULL' and t_i == 'NULL' and t_j == 'NULL': 
        V1 = np.exp(-r*t_k)*np.maximum(S_tk - K, 0)

    if t_k == 'NULL' and t_j == 'NULL' and t_i == 'NULL':
        V1 = np.exp(-r*T)*np.maximum(S[N-1] - K, 0)
        
    #Antithetic variate:
        
    S_a = np.zeros(N)
    S_a[0] = S_0

    ta_i = 'NULL'
    Sa_ti = 'NULL'
    
    ta_j = 'NULL'
    Sa_tj = 'NULL'
    
    ta_k = 'NULL'
    Sa_tk = 'NULL'

    for i in range(0, N_0):

        P_v = np.random.poisson(lam_0*dt, 1)

        if P_v >= 1:
            ta_i = 1
            break

        else:
            S_a[i+1] = S_a[i]*np.exp( (r - 0.5*(sig**2))*dt - sig*Z[i] )


    for i in range(N_0, N-1):

        P = np.random.poisson(lam*dt, 1)

        if P >= 1:
            ta_j = i*dt
            Sa_tj = S_a[i]
            break

        elif S_a[i] > L[i]:
        #elif S_a[i] > K*np.exp(alpha*(i*dt - T)) Uncomment to use K_t barrier
            ta_k = i*dt
            Sa_tk = S_a[i]
            break

        else:
            S_a[i+1] = S_a[i]*np.exp( (r - 0.5*(sig**2))*dt - sig*Z[i] )

    if ta_i != 'NULL' and ta_j == 'NULL' and ta_k == 'NULL':
        V2 = 0

    if ta_j != 'NULL' and ta_i == 'NULL' and ta_k == 'NULL':
        V2 = np.exp(-r*ta_j)*np.maximum(Sa_tj - K, 0)

    if ta_k != 'NULL' and ta_i == 'NULL' and ta_j == 'NULL': 
        V2 = np.exp(-r*ta_k)*np.maximum(Sa_tk - K, 0)

    if ta_k == 'NULL' and ta_j == 'NULL' and ta_i == 'NULL':
        V2 = np.exp(-r*T)*np.maximum(S_a[N-1] - K, 0)

    MC[m] = V1    
    MC_a[m] = 0.5*(V1 + V2)

mean = round(np.mean(MC), 4)
sd = round(np.std(MC), 4)

mean_a = round(np.mean(MC_a), 4)
sd_a = round(np.std(MC_a), 4)

CI = [round(mean - 1.96*sd/np.sqrt(M), 4), round(mean + 1.96*sd/np.sqrt(M), 4)]
CI_a = [round(mean_a - 1.96*sd_a/np.sqrt(M), 4), round(mean_a + 1.96*sd_a/np.sqrt(M), 4)] 

###########################################################
print('Brute-force MC estimate:', mean, CI, sd)
print('Antithetic MC estimate:', mean_a, CI_a, sd_a)

Brute-force MC estimate: 26.0282 [24.397, 27.6594] 26.3186
Antithetic MC estimate: 26.4431 [25.5735, 27.3127] 14.0306
